In [5]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
gt.GTFS_REF.head()

,table,table_required,field,field_required,dtype
0,agency,True,agency_id,False,str
1,agency,True,agency_name,True,str
2,agency,True,agency_url,True,str
3,agency,True,agency_timezone,True,str
4,agency,True,agency_lang,False,str


In [17]:
path = DATA_DIR/'sample_gtfs.zip'
feed = gt.read_gtfs(path, dist_units='km')

# Pick date
date = feed.get_first_week()[0]
print('date', date)


date 20070101


In [28]:
print(range(3))

range(0, 3)


In [27]:
F = feed.copy()
F.routes['route_color'].iat[0] = 'FFF'
f = F.routes.copy()
v = lambda x: pd.isnull(x) or gt.valid_color(x)
cond = ~f['route_color'].map(v)
f[cond]
#gt.check_routes(F)

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,AB,DTA,10,Airport - Bullfrog,NaN,3,NaN,FFF,NaN


In [ ]:
feed.describe(date)

In [ ]:
feed.assess()

In [ ]:
ts = feed.compute_trip_stats()
#ts.to_csv(str(DATA_DIR/'cairns_trip_stats.csv'), index=False)
ts.head()

In [ ]:
rts = feed.compute_route_time_series(ts, date)
gt.downsample(rts, freq='12H')

In [ ]:
trip_id = feed.trips['trip_id'].iat[0]
geo = feed.trip_to_geojson(trip_id, include_stops=True)
path = DATA_DIR/'trip_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
route_id = feed.routes['route_id'].iat[0]
geo = feed.route_to_geojson(route_id, include_stops=True)
path = DATA_DIR/'route_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
# Add distances to feed
#
trip_stats = feed.compute_trip_stats(compute_dist_from_shapes=True)
feed = feed.append_dist_to_stop_times(trip_stats)
feed.stop_times.T

In [ ]:
trip_stats = feed.compute_trip_stats()
route_stats = feed.compute_route_stats(trip_stats, date)
route_stats


In [ ]:
a = pd.Series([np.nan, np.nan])
a.dropna().max()
#gt.get_trips(feed, date, "07:30:00")


In [ ]:

# Pick screen line
path = DATA_DIR/'cairns_screen_line.geojson'
with path.open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])

g = feed.compute_screen_line_counts(line, date)
g

In [ ]:
g.groupby(['route_id', 'orientation']).count()